This  takes input of a given user, date, and radius (default 6 months), and returns relevant information of all activity by that user which occured within that radius of that date.

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tweepy
import datetime
import json

#API key
ck = 'qoMJeRwxZ5dSbgcIPXstbp0Db'
#API secret key
cs = '2j3QDpT9sOQJWpfc1C4Jd9izU0oVXGklMhKdgNqCiXR2gJhSRg'
#Access token
at = '2924992611-ZSPxIjMDld4pMzv46E7eKXQDsYscQME16ffo6MZ'
#Access Token Secret
ats = 'gSJFdAkMb5Ypd4e82lREV2jqUwfH5gJrcFob3nCzphTfv'

# create OAuthHandler object
authenticator = tweepy.OAuthHandler(ck, cs)
# set access token and secret
authenticator.set_access_token(at, ats)
# create tweepy API object to fetch tweets
API = tweepy.API(authenticator, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [51]:
def get_frequency(user, date, radius=datetime.timedelta(days=180)):
    # Make sure we're finding the right user
    if type(user)==str:
        user = API.get_user(screen_name=user).id
    
    datet = datetime.datetime(int(date[0:4]),int(date[5:7]),int(date[8:10]),int(date[11:13]),int(date[14:16]),int(date[17:19]))
    
    old_cursor = 0
    cursor = -1
    
    #find the later endpoint
    tweet_date = datetime.datetime.now()
    while tweet_date > datet+radius and cursor!=old_cursor:
        cursed_activity = API.user_timeline(user_id=user, count=200, cursor=cursor)
        for tweet in cursed_activity:
            tweet_date_str = str(tweet.created_at)
            tweet_date = datetime.datetime(int(tweet_date_str[0:4]),int(tweet_date_str[5:7]),int(tweet_date_str[8:10]),int(tweet_date_str[11:13]),int(tweet_date_str[14:16]),int(tweet_date_str[17:19]))
            old_cursor = cursor
            cursor = tweet.id
            if tweet_date <= datet+radius:
                break
        
    tweets = []

    while tweet_date >= datet-radius and cursor!=old_cursor:
        cursed_activity = API.user_timeline(user_id=user, count=200, cursor=cursor)
        for tweet in cursed_activity:
            tweet_date_str = str(tweet.created_at)
            tweet_date = datetime.datetime(int(tweet_date_str[0:4]),int(tweet_date_str[5:7]),int(tweet_date_str[8:10]),int(tweet_date_str[11:13]),int(tweet_date_str[14:16]),int(tweet_date_str[17:19]))
            old_cursor = cursor
            cursor = tweet.id
            if tweet_date >= datet-radius:
                tweets.append(tweet)
            else: break
        
    
    # Make a list of the statuses
    activiray = []
    #and add the relevant data
    for tweet in tweets:
        try: 
            tweet.retweeted_status
            is_retweet = True
        except: 
            is_retweet = False
        activiray.append([tweet.text,
                         tweet.id,
                         is_retweet,
                         tweet.is_quote_status,
                         type(tweet.in_reply_to_status_id)==int,
                         tweet.retweet_count,
                         tweet.favorite_count,
                         tweet.created_at])
        
    # turn it into a dataframe
    return pd.DataFrame(activiray, columns=['Text', 'Id', 'Is_retweet', 'Is_quote', 'Is_reply', 'Retweets', 'Favorites', 'Created'])

In [52]:
get_frequency('TomiLahren', "2018-07-28T19:21:34.000Z")

Rate limit reached. Sleeping for: 292
Rate limit reached. Sleeping for: 289


KeyboardInterrupt: 